# Libraries

In [1]:
import pandas as pd
import numpy as np
import folium

- Run DBSCAN on the crash rate dataset. This dataset will have coordinates that are closer to LA downtown and other common streets. 
- Integrate the crash rate dataset and the traffic dataset based on the closest distance. Any datapoint closest to 

# Traffic Dataset

In [2]:
df_traffic = pd.read_csv(r'../data/dataProcessing_files/df_modified_ver5.csv')
df_traffic

,DR Number,Area ID,Area Name,Premise Description,Address,Cross Street,month,day,year,hour,Day Category,Latitude,Longitude,Date,Day
0,100100007,1,Central,STREET,PICO,HOPE,11,8,2010,22,night,34.0395,-118.2656,2010-11-8,Monday
1,100100767,1,Central,STREET,1300 N BROADWAY,NaN,3,31,2010,4,early_morning,34.0695,-118.2324,2010-3-31,Wednesday
2,100100831,1,Central,STREET,PICO,CHERRY,4,18,2010,1,early_morning,34.0424,-118.2718,2010-4-18,Sunday
3,100101004,1,Central,STREET,FIGUEROA,WASHINGTON BL,6,5,2010,20,night,34.0344,-118.2714,2010-6-5,Saturday
4,100101322,1,Central,STREET,COLLEGE,ALAMEDA,8,7,2010,16,evening,34.0637,-118.2361,2010-8-7,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570367,222104843,21,Topanga,STREET,WINNETKA AV,VICTORY BL,1,26,2022,19,night,34.1857,-118.5710,2022-1-26,Wednesday
570368,222104874,21,Topanga,STREET,TOPANGA CANYON BL,VANOWEN ST,1,24,2022,15,afternoon,34.1938,-118.6059,2022-1-24,Monday
570369,222104886,21,Topanga,STREET,DE SOTO AV,VENTURA FY,1,24,2022,17,evening,34.1740,-118.5888,2022-1-24,Monday
570370,222104962,21,Topanga,STREET,AVENUE SAN LUIS,DUNMAN AV,1,28,2022,19,night,34.1624,-118.6285,2022-1-28,Friday


In [3]:
df_traffic = df_traffic[['DR Number', 'Address', 'hour', 'Latitude', 'Longitude', 'Day']]
df_traffic.head()

,DR Number,Address,hour,Latitude,Longitude,Day
0,100100007,PICO,22,34.0395,-118.2656,Monday
1,100100767,1300 N BROADWAY,4,34.0695,-118.2324,Wednesday
2,100100831,PICO,1,34.0424,-118.2718,Sunday
3,100101004,FIGUEROA,20,34.0344,-118.2714,Saturday
4,100101322,COLLEGE,16,34.0637,-118.2361,Saturday


In [4]:
df_traffic.rename(columns={
    'Address': 'accident_address',
    'Latitude': 'accident_latitude',
    'Longitude': 'accident_longitude',
    'Day': 'day_week'
}, inplace=True)

/var/folders/gc/h6mhk26x34vc_b1d1db5rxh40000gn/T/ipykernel_18043/3399690453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traffic.rename(columns={


In [7]:
df_traffic['accident_address'] = df_traffic['accident_address'].apply(lambda x: " ".join(x.split()))
df_traffic['accident_address'] = df_traffic['accident_address'].apply(lambda x: x+', Los Angeles')
df_traffic

/var/folders/gc/h6mhk26x34vc_b1d1db5rxh40000gn/T/ipykernel_13014/1421901114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traffic['accident_address'] = df_traffic['accident_address'].apply(lambda x: " ".join(x.split()))
/var/folders/gc/h6mhk26x34vc_b1d1db5rxh40000gn/T/ipykernel_13014/1421901114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traffic['accident_address'] = df_traffic['accident_address'].apply(lambda x: x+', Los Angeles')


,DR Number,accident_address,hour,accident_latitude,accident_longitude,day_week
0,100100007,"PICO, Los Angeles",22,34.0395,-118.2656,Monday
1,100100767,"1300 N BROADWAY, Los Angeles",4,34.0695,-118.2324,Wednesday
2,100100831,"PICO, Los Angeles",1,34.0424,-118.2718,Sunday
3,100101004,"FIGUEROA, Los Angeles",20,34.0344,-118.2714,Saturday
4,100101322,"COLLEGE, Los Angeles",16,34.0637,-118.2361,Saturday
...,...,...,...,...,...,...
570367,222104843,"WINNETKA AV, Los Angeles",19,34.1857,-118.5710,Wednesday
570368,222104874,"TOPANGA CANYON BL, Los Angeles",15,34.1938,-118.6059,Monday
570369,222104886,"DE SOTO AV, Los Angeles",17,34.1740,-118.5888,Monday
570370,222104962,"AVENUE SAN LUIS, Los Angeles",19,34.1624,-118.6285,Friday


In [8]:
latlon = df_traffic[['accident_latitude', 'accident_longitude']].iloc[1:500, :].values.tolist()
mapit = folium.Map(location=[34.0395, -118.2656], zoom_start=6)
for coord in latlon:
    folium.Marker(location=[coord[0], coord[1]], fill_color='#43d9de', radius=8).add_to(mapit)

mapit.save('map.html')

# Crash Rate Dataset with K-Means labels

In [8]:
df_crash = pd.read_csv(r'../data/dataProcessing_files/kmeans_k=25.csv')
df_crash

,street,traffic volume,accident count,crash_rate,latitude,longitude,risk level,Frequent_Day,Frequent_Hour,label
0,"102ND ST, Los Angeles",2238,184,8.22,33.944103,-118.254688,low,Friday,14,18
1,"103RD ST, Los Angeles",138801,531,0.38,33.942642,-118.243168,low,Tuesday,14,18
2,"104TH ST, Los Angeles",17741,239,1.35,33.941790,-118.369982,low,Saturday,17,5
3,"105TH ST, Los Angeles",14131,178,1.26,34.750118,-118.316244,low,Tuesday,1,13
4,"10TH AV, Los Angeles",3642,145,3.98,34.037828,-118.327656,low,Saturday,15,8
...,...,...,...,...,...,...,...,...,...,...
1019,"YOLANDA AV, Los Angeles",24334,76,0.31,34.195669,-118.540414,low,Monday,7,16
1020,"YORK BL, Los Angeles",271200,158,0.06,33.957271,-118.050991,low,Monday,18,14
1021,"YUCCA ST, Los Angeles",22468,204,0.91,34.103720,-118.324950,low,Monday,6,20
1022,"ZELZAH AV, Los Angeles",96051,110,0.11,34.243016,-118.523271,low,Monday,8,1


In [9]:
df_crash

,street,traffic volume,accident count,crash_rate,latitude,longitude,risk level,Frequent_Day,Frequent_Hour,label
0,"102ND ST, Los Angeles",2238,184,8.22,33.944103,-118.254688,low,Friday,14,18
1,"103RD ST, Los Angeles",138801,531,0.38,33.942642,-118.243168,low,Tuesday,14,18
2,"104TH ST, Los Angeles",17741,239,1.35,33.941790,-118.369982,low,Saturday,17,5
3,"105TH ST, Los Angeles",14131,178,1.26,34.750118,-118.316244,low,Tuesday,1,13
4,"10TH AV, Los Angeles",3642,145,3.98,34.037828,-118.327656,low,Saturday,15,8
...,...,...,...,...,...,...,...,...,...,...
1019,"YOLANDA AV, Los Angeles",24334,76,0.31,34.195669,-118.540414,low,Monday,7,16
1020,"YORK BL, Los Angeles",271200,158,0.06,33.957271,-118.050991,low,Monday,18,14
1021,"YUCCA ST, Los Angeles",22468,204,0.91,34.103720,-118.324950,low,Monday,6,20
1022,"ZELZAH AV, Los Angeles",96051,110,0.11,34.243016,-118.523271,low,Monday,8,1


In [10]:
df_crash_streets = df_crash['street'].values.tolist()
df_crash_streets

['102ND ST, Los Angeles',
 '103RD ST, Los Angeles',
 '104TH ST, Los Angeles',
 '105TH ST, Los Angeles',
 '10TH AV, Los Angeles',
 '10TH ST, Los Angeles',
 '111TH ST, Los Angeles',
 '112TH ST, Los Angeles',
 '114TH ST, Los Angeles',
 '115TH ST, Los Angeles',
 '116TH ST, Los Angeles',
 '118 FWY, Los Angeles',
 '118TH ST, Los Angeles',
 '11TH AV, Los Angeles',
 '11TH ST, Los Angeles',
 '12TH ST, Los Angeles',
 '134 FWY, Los Angeles',
 '13TH ST, Los Angeles',
 '14TH ST, Los Angeles',
 '155TH ST, Los Angeles',
 '157TH ST, Los Angeles',
 '15TH ST, Los Angeles',
 '161ST ST, Los Angeles',
 '168TH ST, Los Angeles',
 '16TH ST, Los Angeles',
 '170TH ST, Los Angeles',
 '17TH ST, Los Angeles',
 '186TH ST, Los Angeles',
 '18TH ST, Los Angeles',
 '1ST ST, Los Angeles',
 '20TH ST, Los Angeles',
 '214TH ST, Los Angeles',
 '218TH ST, Los Angeles',
 '21ST ST, Los Angeles',
 '223RD ST, Los Angeles',
 '22ND ST, Los Angeles',
 '23RD ST, Los Angeles',
 '240TH ST, Los Angeles',
 '24TH ST, Los Angeles',
 '253R

In [11]:
df_crash = df_crash[['street', 'traffic volume', 'accident count', 'crash_rate', 'latitude', 'longitude', 'label']]
df_crash.rename(columns={
    'traffic volume': 'traffic_volume',
    'accident count': 'accident_count'
}, inplace=True)
df_crash

/var/folders/gc/h6mhk26x34vc_b1d1db5rxh40000gn/T/ipykernel_13014/668033708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crash.rename(columns={


,street,traffic_volume,accident_count,crash_rate,latitude,longitude,label
0,"102ND ST, Los Angeles",2238,184,8.22,33.944103,-118.254688,18
1,"103RD ST, Los Angeles",138801,531,0.38,33.942642,-118.243168,18
2,"104TH ST, Los Angeles",17741,239,1.35,33.941790,-118.369982,5
3,"105TH ST, Los Angeles",14131,178,1.26,34.750118,-118.316244,13
4,"10TH AV, Los Angeles",3642,145,3.98,34.037828,-118.327656,8
...,...,...,...,...,...,...,...
1019,"YOLANDA AV, Los Angeles",24334,76,0.31,34.195669,-118.540414,16
1020,"YORK BL, Los Angeles",271200,158,0.06,33.957271,-118.050991,14
1021,"YUCCA ST, Los Angeles",22468,204,0.91,34.103720,-118.324950,20
1022,"ZELZAH AV, Los Angeles",96051,110,0.11,34.243016,-118.523271,1


In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_std = scaler.fit_transform(np.array(df_crash['crash_rate']).reshape(-1,1))
x_std.min()

-0.13298769458962886

In [13]:
x_std.mean()

-1.0408340855860843e-17

In [14]:
mean_trafficVol_label = df_crash.groupby('label')['traffic_volume'].mean().to_dict()
mean_accidentVol_label = df_crash.groupby('label')['accident_count'].mean().to_dict()
mean_crashRate_label = df_crash.groupby('label')['crash_rate'].mean().to_dict()
mean_crashRate_label

# print(f'Crash rates below 1% (low): {len(df_crash[df_crash["crash_rate"] <= 1])}')
# print(f'Crash rates between 1%-20% (medium): {len(df_crash[(df_crash["crash_rate"] > 1)&(df_crash["crash_rate"] <= 20)])}')
# print(f'Crash rates above 20% (high): {len(df_crash[df_crash["crash_rate"] > 20])}')

{0: 6.74523076923077,
 1: 6.193846153846154,
 2: 2.702857142857143,
 3: 2.425,
 4: 7.536744186046511,
 5: 4.4373469387755105,
 6: 5.89046875,
 7: 5.3161538461538465,
 8: 14.853333333333333,
 9: 46.1,
 10: 10.020000000000001,
 11: 7.639726027397261,
 12: 13.129550561797753,
 13: 5.57,
 14: 2.4856249999999998,
 15: 7.287999999999999,
 16: 78.36551020408164,
 17: 25.903673469387755,
 18: 26.265555555555554,
 19: 70.0240909090909,
 20: 5.42603448275862,
 21: 16.537166666666668,
 22: 18.586521739130436,
 23: 1.7266666666666668,
 24: 0.25}

- We can say anything less than 4% - low
- 4-15 % - medium
- 15% - high

# Integrating the two datasets

In [35]:
# Run a k-means model to predict the labels for each data in the big dataset.
import pickle

coordinates_crash_rate = df_traffic[['accident_latitude', 'accident_longitude']].values.tolist()

model = pickle.load(open("../results/kmeans_1.0.2.pkl", "rb"))
y_pred = [model.predict(np.array([lat, long]).reshape(1, -1))[0] for lat, long in coordinates_crash_rate]

In [16]:
df_traffic['predicted_labels'] = y_pred

/var/folders/gc/h6mhk26x34vc_b1d1db5rxh40000gn/T/ipykernel_13014/2589123422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traffic['predicted_labels'] = y_pred


In [17]:
df_traffic

,DR Number,accident_address,hour,accident_latitude,accident_longitude,day_week,predicted_labels
0,100100007,"PICO, Los Angeles",22,34.0395,-118.2656,Monday,0
1,100100767,"1300 N BROADWAY, Los Angeles",4,34.0695,-118.2324,Wednesday,12
2,100100831,"PICO, Los Angeles",1,34.0424,-118.2718,Sunday,0
3,100101004,"FIGUEROA, Los Angeles",20,34.0344,-118.2714,Saturday,0
4,100101322,"COLLEGE, Los Angeles",16,34.0637,-118.2361,Saturday,12
...,...,...,...,...,...,...,...
570367,222104843,"WINNETKA AV, Los Angeles",19,34.1857,-118.5710,Wednesday,16
570368,222104874,"TOPANGA CANYON BL, Los Angeles",15,34.1938,-118.6059,Monday,16
570369,222104886,"DE SOTO AV, Los Angeles",17,34.1740,-118.5888,Monday,16
570370,222104962,"AVENUE SAN LUIS, Los Angeles",19,34.1624,-118.6285,Friday,16


In [19]:
def risk_level(x):
    if x > 0 and x <=10:
        return 'low'
    elif x > 11 and x <= 20:
        return 'medium'
    else:
        return 'high'

df_traffic['mean_trafficVolume'] = df_traffic['predicted_labels'].apply(lambda x: mean_trafficVol_label[x])
df_traffic['mean_accidentVolume'] = df_traffic['predicted_labels'].apply(lambda x: mean_accidentVol_label[x])
df_traffic['mean_crashRate'] = df_traffic['predicted_labels'].apply(lambda x: mean_crashRate_label[x])
df_traffic['risk_level'] = df_traffic['mean_crashRate'].apply(lambda x: risk_level(x)) 

/var/folders/gc/h6mhk26x34vc_b1d1db5rxh40000gn/T/ipykernel_13014/303275901.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traffic['mean_trafficVolume'] = df_traffic['predicted_labels'].apply(lambda x: mean_trafficVol_label[x])
/var/folders/gc/h6mhk26x34vc_b1d1db5rxh40000gn/T/ipykernel_13014/303275901.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traffic['mean_accidentVolume'] = df_traffic['predicted_labels'].apply(lambda x: mean_accidentVol_label[x])
/var/folders/gc/h6mhk26x34vc_b1d1db5rxh

In [20]:
df_traffic

,DR Number,accident_address,hour,accident_latitude,accident_longitude,day_week,predicted_labels,mean_trafficVolume,mean_accidentVolume,mean_crashRate,risk_level
0,100100007,"PICO, Los Angeles",22,34.0395,-118.2656,Monday,0,204447.984615,268.707692,6.745231,low
1,100100767,"1300 N BROADWAY, Los Angeles",4,34.0695,-118.2324,Wednesday,12,56487.460674,369.303371,13.129551,medium
2,100100831,"PICO, Los Angeles",1,34.0424,-118.2718,Sunday,0,204447.984615,268.707692,6.745231,low
3,100101004,"FIGUEROA, Los Angeles",20,34.0344,-118.2714,Saturday,0,204447.984615,268.707692,6.745231,low
4,100101322,"COLLEGE, Los Angeles",16,34.0637,-118.2361,Saturday,12,56487.460674,369.303371,13.129551,medium
...,...,...,...,...,...,...,...,...,...,...,...
570367,222104843,"WINNETKA AV, Los Angeles",19,34.1857,-118.5710,Wednesday,16,120269.428571,336.224490,78.365510,high
570368,222104874,"TOPANGA CANYON BL, Los Angeles",15,34.1938,-118.6059,Monday,16,120269.428571,336.224490,78.365510,high
570369,222104886,"DE SOTO AV, Los Angeles",17,34.1740,-118.5888,Monday,16,120269.428571,336.224490,78.365510,high
570370,222104962,"AVENUE SAN LUIS, Los Angeles",19,34.1624,-118.6285,Friday,16,120269.428571,336.224490,78.365510,high


In [21]:
df_traffic['risk_level'].value_counts()

low       296605
medium    138907
high      134860
Name: risk_level, dtype: int64

In [22]:
df_traffic['risk_level'].value_counts()

low       296605
medium    138907
high      134860
Name: risk_level, dtype: int64

In [26]:
x = pd.DataFrame(df_traffic.groupby(['day_week', 'hour'])['mean_trafficVolume'].mean().unstack(level=-1).fillna(0))
x.loc[('Monday', 16)]

90070.14235190421

In [25]:
x = pd.DataFrame(df_traffic.groupby(['day_week', 'hour'])['mean_accidentVolume'].mean().unstack(level=-1).fillna(0))
x.loc[('Monday', 16)]

329.42579039398754

In [29]:
coordinates = zip(df_traffic['accident_latitude'], df_traffic['accident_longitude'])
coordinates = list(coordinates)
print(len(coordinates))

570372


In [30]:
df_crash.groupby('label')['traffic_volume'].mean().to_dict()

{0: 204447.9846153846,
 1: 23996.115384615383,
 2: 122322.95238095238,
 3: 122408.375,
 4: 109305.25581395348,
 5: 81862.12244897959,
 6: 30854.75,
 7: 21949.46153846154,
 8: 79831.64912280702,
 9: 34942.5,
 10: 224395.0,
 11: 67354.57534246576,
 12: 56487.4606741573,
 13: 20539.8,
 14: 80048.6875,
 15: 116363.26666666666,
 16: 120269.42857142857,
 17: 52223.244897959186,
 18: 60659.351851851854,
 19: 34786.625,
 20: 68404.94827586207,
 21: 39392.433333333334,
 22: 29290.72463768116,
 23: 712617.3333333334,
 24: 1555578.0}

In [32]:
model.predict(np.array([coordinates[1][0], coordinates[1][1]]).reshape(1, -1))[0]

12

In [33]:
df_traffic.to_csv(r'../data/dataProcessing_files/data_final.csv', index=False)

In [34]:
df_traffic

,DR Number,accident_address,hour,accident_latitude,accident_longitude,day_week,predicted_labels,mean_trafficVolume,mean_accidentVolume,mean_crashRate,risk_level
0,100100007,"PICO, Los Angeles",22,34.0395,-118.2656,Monday,0,204447.984615,268.707692,6.745231,low
1,100100767,"1300 N BROADWAY, Los Angeles",4,34.0695,-118.2324,Wednesday,12,56487.460674,369.303371,13.129551,medium
2,100100831,"PICO, Los Angeles",1,34.0424,-118.2718,Sunday,0,204447.984615,268.707692,6.745231,low
3,100101004,"FIGUEROA, Los Angeles",20,34.0344,-118.2714,Saturday,0,204447.984615,268.707692,6.745231,low
4,100101322,"COLLEGE, Los Angeles",16,34.0637,-118.2361,Saturday,12,56487.460674,369.303371,13.129551,medium
...,...,...,...,...,...,...,...,...,...,...,...
570367,222104843,"WINNETKA AV, Los Angeles",19,34.1857,-118.5710,Wednesday,16,120269.428571,336.224490,78.365510,high
570368,222104874,"TOPANGA CANYON BL, Los Angeles",15,34.1938,-118.6059,Monday,16,120269.428571,336.224490,78.365510,high
570369,222104886,"DE SOTO AV, Los Angeles",17,34.1740,-118.5888,Monday,16,120269.428571,336.224490,78.365510,high
570370,222104962,"AVENUE SAN LUIS, Los Angeles",19,34.1624,-118.6285,Friday,16,120269.428571,336.224490,78.365510,high


- User inputs address
- Address gets converted to coordinates
- Coordinates passes through K-Means saved model to get a predicted label
- Its associated data will be shown in the popup text
- The traffic volume, traffic accidents and labels can already be plotted on the map and on dataframe.
